In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
import torch
from google.colab import drive
drive.mount('/content/drive')
from glob import glob
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torchsummary import summary
from fastai.metrics import accuracy
from fastai.vision import *
import numpy as np
!pip install fft-conv-pytorch
from fft_conv_pytorch import fft_conv, FFTConv2d

Mounted at /content/drive


In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from fastai.vision.data import normalize
path = "/content/drive/MyDrive/Dataset/brain_tumor_dataset_grayscale"
np.random.seed(41)
defaults = SimpleNamespace(cmap='L', return_fig=False, silent=False)
data = ImageDataBunch.from_folder(path, train="Train", valid ="Valid", valid_pct=0.2,ds_tfms=get_transforms(), size=(256,256), bs=16, num_workers=4)
for itemList in ["train_dl", "valid_dl", "fix_dl", "test_dl"]:
    itemList = getattr(data, itemList)
    if itemList: itemList.x.convert_mode = "L"

In [ ]:
print (data.one_batch()[0].shape)
print (data.classes, data.c, len(data.train_ds), len(data.valid_ds))
print (data.train_ds.classes)
print (data.valid_ds.classes)

torch.Size([16, 1, 256, 256])
['no', 'yes'] 2 203 50
['no', 'yes']
['no', 'yes']


With FFT Conv Layers

In [ ]:
def fft_conv_block(ni, nf, size=3, stride=1):
    for_pad = lambda s: s if s > 2 else 3
    return nn.Sequential(
        FFTConv2d(ni, nf, kernel_size=size, stride=stride,
                  padding=(for_pad(size) - 1)//2, bias=False), 
        nn.BatchNorm2d(nf),
        nn.LeakyReLU(negative_slope=0.1, inplace=True)  
    )

def conv_block(ni, nf, size=3, stride=1):
    for_pad = lambda s: s if s > 2 else 3
    return nn.Sequential(
        nn.Conv2d(ni, nf, kernel_size=size, stride=stride,
                  padding=(for_pad(size) - 1)//2, bias=False), 
        nn.BatchNorm2d(nf),
        nn.LeakyReLU(negative_slope=0.1, inplace=True)  
    )

def triple_conv(ni, nf):
    return nn.Sequential(
        conv_block(ni, nf),
        conv_block(nf, ni, size=1),  
        conv_block(ni, nf)
    )
def fft_triple_conv(ni, nf,size):
    return nn.Sequential(
        fft_conv_block(ni, nf,size),
        conv_block(nf, ni, size=1),  
        fft_conv_block(ni, nf, size)
    )
def maxpooling():
    return nn.MaxPool2d(2, stride=2)

In [ ]:
fft_model2 = nn.Sequential(
    fft_conv_block(1, 8, 17),
    maxpooling(),
    conv_block(8, 16),
    maxpooling(),
    triple_conv(16,32),
    maxpooling(),
    triple_conv(32, 64),
    maxpooling(),
    triple_conv(64, 128),
    maxpooling(),
    triple_conv(128, 256),
    conv_block(256, 128, size=1),
    conv_block(128, 256),
    conv_block(256,2),
    nn.Flatten(),
    nn.Linear(338, 2)
)
fft_learn2 = Learner(data, fft_model2, loss_func = nn.CrossEntropyLoss(), metrics=accuracy)
print(fft_learn2.summary())
fft_learn2.fit_one_cycle(100, max_lr=8e-4)

Sequential
Layer (type)         Output Shape         Param #    Trainable 
_FFTConv             [8, 256, 256]        2,312      True      
______________________________________________________________________
BatchNorm2d          [8, 256, 256]        16         True      
______________________________________________________________________
LeakyReLU            [8, 256, 256]        0          False     
______________________________________________________________________
MaxPool2d            [8, 128, 128]        0          False     
______________________________________________________________________
Conv2d               [16, 128, 128]       1,152      True      
______________________________________________________________________
BatchNorm2d          [16, 128, 128]       32         True      
______________________________________________________________________
LeakyReLU            [16, 128, 128]       0          False     
___________________________________________________

epoch,train_loss,valid_loss,accuracy,time
0,0.611912,0.692051,0.620000,00:25
1,0.593843,0.675574,0.620000,00:25
2,0.572410,0.569434,0.660000,00:25
3,0.562331,0.475609,0.840000,00:25
4,0.541103,0.540380,0.820000,00:25
5,0.535528,0.443420,0.820000,00:25
6,0.524893,0.431420,0.840000,00:25
7,0.518056,0.408837,0.820000,00:25
8,0.507023,0.463390,0.800000,00:25
9,0.498167,0.475090,0.760000,00:27
